In [1]:
import cv2
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow import keras
model = keras.models.load_model('model1.h5')

In [3]:
from ipynb.fs.full.HandTrackingModule import HandDetector

In [4]:
class Button():
    def __init__(self,x,y,r,color,text):
        self.x = x
        self.y = y
        self.r = r
        self.color = color
        self.text = text
        
    def draw(self,frame):
        cv2.circle(frame,(self.x,self.y),self.r,self.color,-1)
        cv2.circle(frame,(self.x,self.y),self.r,(0,0,0),2)
        cv2.putText(frame,str(self.text),(self.x-25,self.y+15),7,2,(0,0,0),2)
        
    def isClicked(self,frame,x,y):
        if x >= self.x - self.r and x <= self.x + self.r and y >= self.y - self.r and y <= self.y + self.r :
            return True
        return False
    
    def click(self,frame):
        if self.color == (0,0,200):
            cv2.circle(frame,(self.x,self.y),self.r,(0,0,255),-1)
        elif self.color == (0,200,0):
            cv2.circle(frame,(self.x,self.y),self.r,(0,255,0),-1) 
        elif self.color == (200,0,0):
            cv2.circle(frame,(self.x,self.y),self.r,(255,0,0),-1)    
        cv2.circle(frame,(self.x,self.y),self.r,(0,0,0),2)
        cv2.putText(frame,str(self.text),(self.x-20,self.y+15),7,3,(0,0,0),4)
        
    def getColor(self,frame):
        return self.color
    
    def getText(self,frame):
        return self.text

In [16]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
detector = HandDetector()
buttonTexts = ["R","G","B","B"]
buttonList = []
xOffset = 1200
yOffset = 150
radius = 50
drawColor = (0,0,0)
xp,yp = 0,0
imgCanvas = np.zeros((720,1280,3),np.uint8)
for i,text in enumerate(buttonTexts):
    x = xOffset
    y = yOffset + i*120
    if i==0:
        buttonList.append(Button(x,y,radius,(0,0,200),buttonTexts[i]))
    elif i==1:
        buttonList.append(Button(x,y,radius,(0,200,0),buttonTexts[i]))
    elif i==2:
        buttonList.append(Button(x,y,radius,(200,0,0),buttonTexts[i]))
    else:
        buttonList.append(Button(x,y,radius,(0,0,0),buttonTexts[i]))

#buttonList.append(Button(700,200,50,(255,255,255),"Go"))        
while True:
    success,frame = cap.read()
    if not success:
        break
        
    frame = cv2.flip(frame,1)
    hands,frame = detector.findHands(frame) 
    #cv2.rectangle(frame,(200,200),(600,600),(0,0,0),-1)
    test = frame[200:600,200:600]
    cv2.circle(frame,(1500,350),450,drawColor,10)
   # cv2.circle(frame,(1200,200),50,(0,255,0),-1)
   # cv2.circle(frame,(1200,200),50,(0,0,0),2)
    for button in buttonList:
        button.draw(frame)
    
    
    if hands:
        hand1 = hands[0]
        lmList = hand1["lmList"]
        x1,y1 = lmList[8][0],lmList[8][1]
        x2,y2 = lmList[12][0],lmList[12][1]
        
        fingers1 = detector.fingersUp(hand1)
        
        #selection mode
        if fingers1[1] == 1 and fingers1[2] == 1 :
            xp,yp = 0,0
            cv2.circle(frame,(x1,y1),10,drawColor,-1)
            cv2.circle(frame,(x2,y2),10,drawColor,-1)
            cv2.putText(frame,str("selection mode"),(50,50),5,2,drawColor,2)
            for button in buttonList :
                if button.isClicked(frame,x1,y1):
                    text = button.getText(frame)
                    if text == "Go":
                        test = frame[0:720,0:500]
                        gray = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
                        resized = cv2.resize(gray,(28,28),interpolation = cv2.INTER_AREA)
                        newImg = tf.keras.utils.normalize(resized,axis =1)
                        newImg = np.array(newImg).reshape(-1,28,28,1)
                        pred = model.predict(newImg)
                        print(np.argmax(pred))
                        print((pred))
                        cv2.putText(frame,str(np.argmax(pred)),(200,100),0,1,(255,0,255),2)
                    
                    else:
                        button.click(frame)
                        drawColor = button.getColor(frame)
            #print("selection mode")
        
        #drawing mode
        elif fingers1[1] == 1 and fingers1[2] == 0 :
            #print("drawing mode")
            cv2.putText(frame,str("drawing mode"),(50,50),5,2,drawColor,2)
            if xp == 0 and yp == 0:
                xp = x1
                yp = y1
            if drawColor == (0,0,200):    
                cv2.line(imgCanvas,(x1,y1),(xp,yp),(0,0,255),30)
            elif drawColor == (0,200,0):    
                cv2.line(imgCanvas,(x1,y1),(xp,yp),(0,255,0),30)
            elif drawColor == (200,0,0):    
                cv2.line(imgCanvas,(x1,y1),(xp,yp),(255,0,0),30)
            elif drawColor == (0,0,0):    
                cv2.circle(imgCanvas,(x1,y1),50,(0,0,0),-1) 
                cv2.circle(frame,(x1,y1),50,(0,0,0),-1) 
         
        xp = x1
        yp = y1
    imgGray  = cv2.cvtColor(imgCanvas,cv2.COLOR_RGB2GRAY)
    _,imgInv = cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    frame = cv2.bitwise_and(frame,imgInv)
    frame = cv2.bitwise_or(frame,imgCanvas)
    
    
    test = frame[200:600,200:600]
    cv2.imshow("frame",frame)
   # cv2.imshow("frame1",test)
    #cv2.imshow("canvas",imgCanvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()